In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- This notebook is simple baseline with LGBM
- I used [this note](https://www.kaggle.com/ilialar/ubiquant-eda-and-baseline) as a reference.
- Don't forget to disable Internet access!

# 0.Import libraries and pickle file

- The original data is too large to load, so we will load the Pickle file created in [this note](https://www.kaggle.com/shashimo/ubiquant-how-to-make-pickle-file).

In [ ]:
# Import helpful libraries
import pandas as pd
import pickle
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold 

#Importing pickle files
with open('/kaggle/input/ubiquanttrainpicklefile/train.pickle', 'rb') as f:
    train_full = pickle.load(f)


# 1.Create a dataset that will be used in this analyses

In [ ]:
train_full.shape

In [ ]:
train_full.head()

# 2.LGBM

In [ ]:
seed = 0
folds = 3
models = []
features = [f'f_{i}' for i in range(300)]
target = 'target'

skf = StratifiedKFold(folds, shuffle = True, random_state = seed)

for train_index, test_index in skf.split(train_full, train_full['investment_id']):
    train = train_full.iloc[train_index]
    valid = train_full.iloc[test_index]
    
    lgbm = LGBMRegressor(
        num_leaves=2 ** np.random.randint(3, 8),
        learning_rate = 10 ** (-np.random.uniform(0.1,2)),
        n_estimators = 1000,
        min_child_samples = 1000, 
        subsample=np.random.uniform(0.5,1.0), 
        subsample_freq=1,
        n_jobs= -1
    )

    lgbm.fit(train[features], train[target], eval_set = (valid[features], valid[target]), early_stopping_rounds = 10)
    models.append(lgbm)

In [ ]:
import ubiquant
#env = ubiquant.make_env()   # initialize the environment
#iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    
    test_df['target']  = 0
    
    for lgbm in models:
        test_df['target'] += lgbm.predict(test_df[features])
    test_df['target'] /= len(models)
    env.predict(test_df[['row_id','target']])